In [50]:

import pydotplus
import subprocess

In [186]:
class Morph:
    def __init__(self,surface,base,pos,pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
    def is_end_of_sentence(self):
        return self.pos1 == '句点'
    def __str__(self):
        return 'surface: {},base: {},pos: {},pos1: {}'.format(self.surface,\
                                            self.base, self.pos, self.pos1)
def make_morph_list(analyzed_file_name: str) -> list:
    """
    係り受けした文章ファイル→[[1文morphオブジェクト],[1文morphオブジェクト]...]
    """
    sentences = []
    sentence = []
    with open(analyzed_file_name, encoding='utf-8') as input_file:
        for line in input_file:
            #* 0 1D 0/1 0.000000
             #吾輩    名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ
            line_list=line.split()
            #['吾輩', '名詞,代名詞,一般,*,*,*,吾輩,ワガハイ,ワガハイ']
            if (line_list[0] == '*') | (line_list[0] == 'EOS'):
                pass
            else:
                line_list = line_list[0].split(',') + line_list[1].split(',')
                
                _morph = Morph(surface=line_list[0], base=line_list[7],pos=line_list[1], pos1=line_list[2])
                sentence.append(_morph)
                
                if _morph.is_end_of_sentence():
                    sentences.append(sentence)
                    sentence = []
    return sentences

morphed_sentences = make_morph_list('newkokoro.txt.cabocha')
#[<__main__.Morph object at 0x0000014330DA8C88>, #文ごと
for morph in morphed_sentences[0]:
    print(str(morph))

surface: 先生,base: 先生,pos: 名詞,pos1: 一般
surface: は,base: は,pos: 助詞,pos1: 係助詞
surface: これら,base: これら,pos: 名詞,pos1: 代名詞
surface: の,base: の,pos: 助詞,pos1: 連体化
surface: 墓標,base: 墓標,pos: 名詞,pos1: 一般
surface: が,base: が,pos: 助詞,pos1: 格助詞
surface: 現,base: 現,pos: 接頭詞,pos1: 名詞接続
surface: わす,base: わする,pos: 動詞,pos1: 自立
surface: 人種,base: 人種,pos: 名詞,pos1: 一般
surface: 々,base: 々,pos: 記号,pos1: 一般
surface: の,base: の,pos: 助詞,pos1: 連体化
surface: 様式,base: 様式,pos: 名詞,pos1: 一般
surface: に対して,base: に対して,pos: 助詞,pos1: 格助詞
surface: 、,base: 、,pos: 記号,pos1: 読点
surface: 私,base: 私,pos: 名詞,pos1: 代名詞
surface: ほど,base: ほど,pos: 助詞,pos1: 副助詞
surface: に,base: に,pos: 助詞,pos1: 格助詞
surface: 滑稽,base: 滑稽,pos: 名詞,pos1: 一般
surface: も,base: も,pos: 助詞,pos1: 係助詞
surface: アイロニー,base: アイロニー,pos: 名詞,pos1: 一般
surface: も,base: も,pos: 助詞,pos1: 係助詞
surface: 認め,base: 認める,pos: 動詞,pos1: 自立
surface: て,base: てる,pos: 動詞,pos1: 非自立
surface: ない,base: ない,pos: 助動詞,pos1: *
surface: らしかっ,base: らしい,pos: 助動詞,pos1: *
surface: た,base: た,pos: 助動詞,pos1: *
surfa

In [198]:
class Chunk:
    def __init__(self, morphs:list, dst:str, srcs:str) ->None:
#形態素（Morphオブジェクト）のリスト（morphs），係り先文節インデックス番号（dst），
#係り元文節インデックス番号のリスト（srcs）をメンバ変数に持つ
        self.morphs = morphs
        self.dst = int(dst.strip("D"))
        self.srcs = int(srcs)
    def join_morphs(self) -> str:
        return ''.join([_motph.surface for _motph in self.morphs if _morph.pos != '記号'])
    def __str__(self) ->str:
        return 'srcs: {},dst: {}, morphs: ({})'.format(self.srcs, self.dst,\
                            ' / '.join([str(_morph) for _morph in self.morphs]))
    #morphs当初は空っぽ

def make_chunk_list(analyzed_file_name: str) ->list:
    """
    係り受けした文章ファイル→chunkオブジェクトのリスト
    """
    sentences = []
    sentence = []
    _chunk = None
    count = -1
    with open(analyzed_file_name, encoding='utf-8')as input_file:
         for line in input_file:  
            line_list = line.split()
            if line_list[0] == '*':
                if _chunk is not None:
                    sentence.append(_chunk)
                _chunk = Chunk(morphs=[], dst=line_list[2], srcs=line_list[1])
            elif line_list[0] == 'EOS':
                break
            else:
                #print(line_list)
                line_list = line_list[0].split(',') + line_list[1].split(',')
                #print(line_list)
                _morph = Morph(surface=line_list[0], base=line_list[7], \
                              pos=line_list[1], pos1=line_list[2])
                #print(count,_morph)
                #print(count,_chunk.morphs)
                _chunk.morphs.append(_morph)
                #print(count,_chunk.morphs)
                #if line_list[0] == '。' and input_file[count-1]!='」':
                if line_list[0] == '。':
                    if _chunk is not None:
                        sentence.append(_chunk)
                    if len(sentence) > 0:
                        sentences.append(sentence)
                    _chunk = None
                    sentence = []
                        
    return sentences
chunked_sentences = make_chunk_list('newkokoro.txt.cabocha')
for chunk in chunked_sentences[0]: 
    print(str(chunk))

srcs: 0,dst: 9, morphs: (surface: 先生,base: 先生,pos: 名詞,pos1: 一般 / surface: は,base: は,pos: 助詞,pos1: 係助詞)
srcs: 1,dst: 2, morphs: (surface: これら,base: これら,pos: 名詞,pos1: 代名詞 / surface: の,base: の,pos: 助詞,pos1: 連体化)
srcs: 2,dst: 3, morphs: (surface: 墓標,base: 墓標,pos: 名詞,pos1: 一般 / surface: が,base: が,pos: 助詞,pos1: 格助詞)
srcs: 3,dst: 5, morphs: (surface: 現,base: 現,pos: 接頭詞,pos1: 名詞接続 / surface: わす,base: わする,pos: 動詞,pos1: 自立)
srcs: 4,dst: 5, morphs: (surface: 人種,base: 人種,pos: 名詞,pos1: 一般 / surface: 々,base: 々,pos: 記号,pos1: 一般 / surface: の,base: の,pos: 助詞,pos1: 連体化)
srcs: 5,dst: 9, morphs: (surface: 様式,base: 様式,pos: 名詞,pos1: 一般 / surface: に対して,base: に対して,pos: 助詞,pos1: 格助詞 / surface: 、,base: 、,pos: 記号,pos1: 読点)
srcs: 6,dst: 9, morphs: (surface: 私,base: 私,pos: 名詞,pos1: 代名詞 / surface: ほど,base: ほど,pos: 助詞,pos1: 副助詞 / surface: に,base: に,pos: 助詞,pos1: 格助詞)
srcs: 7,dst: 8, morphs: (surface: 滑稽,base: 滑稽,pos: 名詞,pos1: 一般 / surface: も,base: も,pos: 助詞,pos1: 係助詞)
srcs: 8,dst: 9, morphs: (surface: アイロニー,base: アイ

In [197]:
def is_valid_chunk(_chunk, sentence):
    return _chunk.join

SyntaxError: invalid syntax (<ipython-input-197-4920083e5fc8>, line 1)